In [1]:
import torch
from torch import nn
import torchvision

### 1. Hazır Model ve Scratch Modelin Bir Arada Gösterilmesi

Öncelikli olarak torchvision kütüphanesinden modeli çekelim ve aynısını yazmaya çalışalım

In [2]:
eff_net = torchvision.models.efficientnet_b0(pretrained=True)
eff_net

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 58.1MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

Burada model incelendiğinde Conv2dNormAction sınıfının bloklar içerisinde uygulandığı ve padding None olması durumunda same olarak ayarlandığıdır.

In [ ]:
class Conv2dNormAction(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=None, groups=1) -> None:
        super(Conv2dNormAction, self).__init__()

        # Eğer padding None ise, otomatik olarak "same" padding uygula
        if padding is None:
            padding = (kernel_size - 1) // 2  # 'same' padding mantığı

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, groups=groups, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.SiLU(inplace=True)
        )

    def forward(self, x) -> torch.Tensor:
        return self.conv_block(x)

Buradaki önemli bir diğer fark da SqueezeExcitation sınıfındaki silu aktivasyonudur. Burada geleneksel yaklaşım olan relu yerine silu kullanılarak nöronlarda yaşanan dead neuron probleminin ortadan kalkması amaçlanmıştır.

In [ ]:
class SqueezeExcitation(nn.Module):
  def __init__(self, in_channels, reduced_dim) -> None:
    super(SqueezeExcitation, self).__init__()
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.conv1 = nn.Conv2d(in_channels, reduced_dim, kernel_size = 1)
    self.silu = nn.SiLU(inplace = True)
    self.conv2 = nn.Conv2d(reduced_dim, in_channels, kernel_size = 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x) -> torch.Tensor:
    scale = self.pool(x)
    scale = self.conv1(scale)
    scale = self.silu(scale)
    scale = self.conv2(scale)
    scale = self.sigmoid(scale)
    return x * scale

In [ ]:
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, expand_ratio, reduction=4, survival_prob=0.8) -> None:
        super(MBConv, self).__init__()
        self.survival_prob = survival_prob
        #Eğer girdi ve çıktı kanalları eşitse ve stride 1 ise, residual bağlantı kullan
        self.use_residual = in_channels == out_channels and stride == 1
        hidden_dim = int(in_channels * expand_ratio)
        reduced_dim = in_channels // reduction
        # Girdi ve çıktı kanalları farklıysa, genişletme katmanı ekle
        self.expand = in_channels != hidden_dim
        if self.expand:
            self.expand_conv = Conv2dNormAction(in_channels, hidden_dim, kernel_size=1, stride=1)

        self.block = nn.Sequential(
            Conv2dNormAction(hidden_dim, hidden_dim, kernel_size, stride, groups=hidden_dim),
            SqueezeExcitation(hidden_dim, reduced_dim),
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x) -> torch.Tensor:
        identity = x  # Residual bağlantı için sakla
        if self.expand:
            x = self.expand_conv(x)

        out = self.block(x)

        if self.use_residual:
            if self.training and self.survival_prob < 1:
                if torch.rand(1).item() < self.survival_prob:
                    out = identity + out
            else:
                out = identity + out

        return out


In [ ]:
class EfficientNet(nn.Module):
    def __init__(self, num_classes=1000, width_mult=1.0, depth_mult=1.0, survival_prob=0.8) -> None:
        super(EfficientNet, self).__init__()
        # EfficientNet B0-B7 arası için MBConv blok parametreleri
        block_args = [
            (32, 16, 3, 1, 1, 1),  
            (16, 24, 3, 2, 6, 2),  
            (24, 40, 5, 2, 6, 2),
            (40, 80, 3, 2, 6, 3),
            (80, 112, 5, 1, 6, 3),
            (112, 192, 5, 2, 6, 4),
            (192, 320, 3, 1, 6, 1),
        ]

        # Genişlik ve Derinlik Faktörlerini Uygula
        block_args = [
            (int(in_c * width_mult), int(out_c * width_mult), k, s, e, int(n * depth_mult))
            for in_c, out_c, k, s, e, n in block_args
        ]

        self.stem = Conv2dNormAction(3, block_args[0][0], kernel_size=3, stride=2)

        layers = []
        for in_c, out_c, k, s, e, n in block_args:
          for i in range(n):
            stride = s if i == 0 else 1  # İlk blok stride uygular, diğerleri 1 olur.
            layers.append(MBConv(in_c, out_c, k, stride, e, survival_prob=survival_prob))
            in_c = out_c  # Bir sonraki blok çıkış kanalını giriş olarak kullanır.

        self.blocks = nn.Sequential(*layers)

        self.head = nn.Sequential(
            Conv2dNormAction(block_args[-1][1], 1280, kernel_size=1),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(1280, num_classes)
        )

    def forward(self, x) -> torch.Tensor:
        x = self.stem(x)
        x = self.blocks(x)
        x = self.head(x)
        return x

# Test
model = EfficientNet(num_classes=10)
x = torch.randn(1, 3, 224, 224)
output = model(x)
print(output.shape)  # torch.Size([1, 10]) olmalı


torch.Size([1, 10])


In [32]:
model

EfficientNet(
  (stem): Conv2dNormAction(
    (conv_block): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
  )
  (blocks): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): Conv2dNormAction(
          (conv_block): Sequential(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (1): SqueezeExcitation(
          (pool): AdaptiveAvgPool2d(output_size=1)
          (conv1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (silu): SiLU(inplace=True)
          (conv2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (sigmoid): Sigmoid()
        )
        (2): Con